# Analyzing CTCAC tax credits project applications from previous years 
Context: 2025 4% allocations. After training on round 1 data, the model is used to predict the round 2 (I did this manually), but only achieved 60% accuracy (terrible result)  
Testing on round 2 data shows that our model trained on only round 1 data is insufficient.   
Possible solutions:
- Get more training data from previous years (get at least 1000 data points if possible)
- include fund allocation per round data into the training 

Therefore, this notebook will be used to explore possible solutions to this problem


In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# ensure changes in files are reflected
%load_ext autoreload
%autoreload 2
# Set display options to show all columns
pd.set_option('display.max_columns', None)  # Show all columns

In [16]:
# 2025 DataFrames
R1_2025_applicant = pd.read_excel("../data/external/2025-R1-ApplicantList.xlsx", header=1, index_col=None)
R1_2025_applicant.attrs['file_name'] = "../data/external/2025-R1-ApplicantList.xlsx"  # Corrected to match file
award_2025_R1 = pd.read_excel("../data/external/2025-R1-AwardList.xlsx")
award_2025_R1.attrs['file_name'] = "../data/external/2025-R1-AwardList.xlsx"

# 2024 DataFrames
R1_2024_applicant = pd.read_excel("../data/external/2024-R1-ApplicantList.xlsx", header=1, index_col=None)
R1_2024_applicant.attrs['file_name'] = "../data/external/2024-R1-ApplicantList.xlsx"  # Corrected to match file
R2_2024_applicant = pd.read_excel("../data/external/2024-R2-ApplicantList.xlsx", header=1, index_col=None)
R2_2024_applicant.attrs['file_name'] = "../data/external/2024-R2-ApplicantList.xlsx"  # Corrected to match file
award_2024 = pd.read_excel("../data/external/2024-Financing-data.xlsx", index_col=None)
award_2024.attrs['file_name'] = "../data/external/2024-Financing-data.xlsx"

# 2023 DataFrames (completing the missing attrs assignments)
R1_2023_applicant = pd.read_excel("../data/external/2023-R1-ApplicantList.xlsx", header=1, index_col=None)
R1_2023_applicant.attrs['file_name'] = "../data/external/2023-R1-ApplicantList.xlsx"
R2_2023_applicant = pd.read_excel("../data/external/2023-R2-ApplicantList.xlsx", header=1, index_col=None)
R2_2023_applicant.attrs['file_name'] = "../data/external/2023-R2-ApplicantList.xlsx"
R3_2023_applicant = pd.read_excel("../data/external/2023-R3-ApplicantList.xlsx", header=1, index_col=None)
R3_2023_applicant.attrs['file_name'] = "../data/external/2023-R3-ApplicantList.xlsx"
award_2023 = pd.read_excel("../data/external/2023-Financing-data.xlsx", index_col=None)
award_2023.attrs['file_name'] = "../data/external/2023-Financing-data.xlsx"


# List of all DataFrames with their names for printing
dataframes = [
    (R1_2023_applicant, "R1_2023_applicant"),
    (R2_2023_applicant, "R2_2023_applicant"),
    (R3_2023_applicant, "R3_2023_applicant"),
    (award_2023, "award_2023"),
    (R1_2024_applicant, "R1_2024_applicant"),
    (R2_2024_applicant, "R2_2024_applicant"),
    (award_2024, "award_2024"),
    (R1_2025_applicant, "R1_2025_applicant"),
    (award_2025_R1, "award_2025_R1")
]

# Print file name and columns for each DataFrame
for df, df_name in dataframes:
    print(f"DataFrame: {df_name}")
    print(f"File Name: {df.attrs['file_name']}")
    print("Columns:", df.columns.tolist())
    # print("info:", df.describe())
    print("-" * 50)

DataFrame: R1_2023_applicant
File Name: ../data/external/2023-R1-ApplicantList.xlsx
Columns: ['APPLICATION NUMBER', 'PROJECT NAME', 'CONSTRUCTION TYPE', 'HOUSING TYPE', 'CITY', 'COUNTY', 'TOTAL UNITS', 'LOW INCOME UNITS', 'MARKET RATE UNITS', 'UNITS FOR HOMELESS INDIVIDUALS', 'AVERAGE AFFORDABILTY (TARGETED AMI)', 'TOTAL PROJECT COST', 'CONSTRUCTION BOND TAX-EXEMPT FINANCING AMOUNT', 'ANNUAL FEDERAL CREDIT REQUESTED', 'TOTAL STATE CREDIT REQUESTED', 'CDLAC NON-GEOGRAPHIC POOL ', 'NEW CONSTRUCTION SET ASIDES', 'SECONDARY NEW CONSTRUCTION SET ASIDE IF APPLICABLE', 'TCAC GEOGRAPHIC REGION', 'CDLAC GEOGRAPHIC REGION', 'CDLAC TOTAL POINTS SCORE', 'PRESERVATION AND OTHER REHAB. PROJECT PRIORITIES (20 PTS)', 'NEW CONSTRUCTION DENSITY & LOCAL INCENTIVES (10 PTS)', 'EXCEEDING MINIMUM INCOME RESTRICTIONS (20 PTS)', 'EXCEEDING MINIMUM RENT RESTRICTIONS (10 PTS)', 'GP & MGMT. CO. EXPERIENCE (10 PTS)', 'HOUSING NEEDS (10 PTS)', 'LEVERAGED SOFT RESOURCES (8 PTS)', 'READINESS TO PROCEED (10 PTS)', 'A

In [68]:
import re 
# Function to find columns by partial name match (case-insensitive)
def find_columns(df, pattern):
    """
    Find columns in a DataFrame that match a given pattern (case-insensitive).
    pattern: str, e.g., 'id', 'name', 'status'
    Returns: list of matching column names
    """
    return [col for col in df.columns if re.search(pattern, col, re.IGNORECASE)]

# Function to summarize columns with describe()
def summarize_columns(df, df_name, file_name, patterns):
    """
    Run describe() on columns matching the given patterns.
    patterns: list of strings to match column names (e.g., ['id', 'name'])
    """
    print(f"DataFrame: {df_name}")
    print(f"File Name: {file_name}")
    
    for pattern in patterns:
        matched_columns = find_columns(df, pattern)
        if not matched_columns:
            print(f"No columns found matching pattern '{pattern}'")
            continue
        
        print(f"\nPattern: {pattern}")
        for col in matched_columns:
            print(f"\nColumn: {col}")
            # Check if column is categorical or object
            if df[col].dtype in ['category', 'object']:
                print(df[col].describe())
                print(df[col].value_counts())
            print("-" * 50)
    print("=" * 50)

# List of patterns to match column names (modify based on your data)
patterns = ['bond']  # Adjust these based on expected column names

# Loop through DataFrames and summarize matched columns
for df, df_name in dataframes:
    summarize_columns(df, df_name, df.attrs['file_name'], patterns)

DataFrame: R1_2025_applicant
File Name: ../data/external/2025-R1-ApplicantList.xlsx

Pattern: bond

Column: BOND REQUEST
--------------------------------------------------
DataFrame: award_2025_R1
File Name: ../data/external/2025-R1-AwardList.xlsx

Pattern: bond

Column: CONSTRUCTION BOND TAX-EXEMPT FINANCING AMOUNT
--------------------------------------------------
DataFrame: R1_2024_applicant
File Name: ../data/external/2024-R1-ApplicantList.xlsx

Pattern: bond

Column: CONSTRUCTION BOND TAX-EXEMPT FINANCING AMOUNT
--------------------------------------------------
DataFrame: R2_2024_applicant
File Name: ../data/external/2024-R2-ApplicantList.xlsx

Pattern: bond

Column: Bond Request
--------------------------------------------------
DataFrame: award_2024
File Name: ../data/external/2024-Financing-data.xlsx
No columns found matching pattern 'bond'
DataFrame: R1_2023_applicant
File Name: ../data/external/2023-R1-ApplicantList.xlsx

Pattern: bond

Column: CONSTRUCTION BOND TAX-EXEMPT F

## Issue
The columns and their names are all slightly different from one another. How do I merge them into a single file? What about future excel files? Is it possible to setup a pipeline that deals with all these variability? 

In [41]:
import re

for df in df_list: 
    print(df.shape)
    cols_to_drop = [col for col in df.columns if (re.match(r'GP.*', col) or re.match(r'(.* PTS)', col) or  re.match(r'Points,.*', col))]
    print(cols_to_drop)
    df = df.drop(columns=cols_to_drop)
    print("new df shape: ", df.shape)
    print(df.columns)
    print("-----------------\n")

(161, 45)
['PRESERVATION AND OTHER REHAB. PROJECT PRIORITIES (20 PTS)', 'NEW CONSTRUCTION DENSITY & LOCAL INCENTIVES (10 PTS)', 'EXCEEDING MINIMUM INCOME RESTRICTIONS (20 PTS)', 'EXCEEDING MINIMUM RENT RESTRICTIONS (10 PTS)', 'GP & MGMT. CO. EXPERIENCE (10 PTS)', 'HOUSING NEEDS (10 PTS)', 'LEVERAGED SOFT RESOURCES (8 PTS)', 'READINESS TO PROCEED (10 PTS)', 'AFFIRMATIVELY FURTHERING FAIR HOUSING (10 PTS)', 'SERVICE AMENITIES (10 PTS)', 'COST CONTAINMENT (12 PTS)', 'SITE AMENITIES (10 PTS)', 'GP 1 COMPANY NAME', 'GP 1 CONTACT NAME', 'GP 1 PARENT COMPANY', 'GP 2 COMPANY NAME', 'GP 2 CONTACT NAME', 'GP 2 PARENT COMPANY', 'GP 3 COMPANY NAME', 'GP 3 CONTACT NAME', 'GP 3 PARENT COMPANY']
new df shape:  (161, 24)
Index(['APPLICATION NUMBER', 'PROJECT NAME', 'CONSTRUCTION TYPE',
       'HOUSING TYPE', 'CITY', 'COUNTY', 'TOTAL UNITS', 'LOW INCOME UNITS',
       'MARKET RATE UNITS', 'UNITS FOR HOMELESS INDIVIDUALS',
       'AVERAGE AFFORDABILTY (TARGETED AMI)', 'TOTAL PROJECT COST',
       'CONST

Since there are so many columns, I focused on 10 of them:
- avg_targeted_affordability
- CDLAC_total_points_score
- CDLAC_tie_breaker_self_score
- bond_request_amount
- homeless_percent
- construction_type
- housing_type
- CDLAC_pool_type
- new_construction_set_aside
- CDLAC_region


# Column Name Variations by Year

The table below shows the variations in column names for the numeric features

| Dataset                | AVERAGE TARGETED AFFORDABILITY         | CDLAC TIE-BREAKER SELF SCORE | CDLAC TOTAL POINTS SCORE | BOND REQUEST                              | UNITS FOR HOMELESS INDIVIDUALS |
|------------------------|----------------------------------------|------------------------------|--------------------------|-------------------------------------------|--------------------------------|
| R1_2023_applicant      | AVERAGE AFFORDABILTY (TARGETED AMI)    | CDLAC TIE-BREAKER SELF SCORE | CDLAC TOTAL POINTS SCORE | CONSTRUCTION BOND TAX-EXEMPT FINANCING AMOUNT | UNITS FOR HOMELESS INDIVIDUALS |
| R2_2023_applicant      | AVERAGE AFFORDABILTY (TARGETED AMI)    | CDLAC TIE-BREAKER SELF SCORE | CDLAC TOTAL POINTS SCORE | CONSTRUCTION BOND TAX-EXEMPT FINANCING AMOUNT | UNITS FOR HOMELESS INDIVIDUALS |
| R3_2023_applicant      | AVERAGE AFFORDABILTY (TARGETED AMI)    | CDLAC TIE-BREAKER SELF SCORE | CDLAC TOTAL POINTS SCORE | CONSTRUCTION BOND TAX-EXEMPT FINANCING AMOUNT | UNITS FOR HOMELESS INDIVIDUALS |
| R1_2024_applicant      | AVERAGE AFFORDABILTY (TARGETED AMI)    | CDLAC TIE-BREAKER SELF SCORE | CDLAC TOTAL POINTS SCORE | CONSTRUCTION BOND TAX-EXEMPT FINANCING AMOUNT | UNITS FOR HOMELESS INDIVIDUALS |
| R2_2024_applicant      | Average Targeted Affordability         | Tie-Breaker Self Score       | CDLAC Total Points       | Bond Request                              | Units for Homeless             |
| R1_2025_applicant      | AVERAGE TARGETED AFFORDABILITY         | CDLAC TIE-BREAKER SELF SCORE | CDLAC TOTAL POINTS SCORE | BOND REQUEST                              | UNITS FOR HOMELESS             |
> note that starting 2024 R2, the names started to drift 


As for categorical features, 

- Construction type has 4 possible types: New Construction, Adaptive Reuse, Rehabilitation-Only and "Acq and Rehabilitation"
- Housing type has 6 possible types: Large Family, Senior, Special Needs, At-Risk, SRO, Mixed Housing 

| Dataset | CDLAC Pool Type | New Construction Set Aside |
| ------- | --------------- | -------------------------- |
| Pior to R2 2024 | one column; 5 types (new construction, Rehabilitation, Rural, Preservation and BIPOC) | two columns, one for primary and one for secondary |
| After R2 2024 | split into two columns; one with 4 types + a new BIPOC column |  combined into one column |



In [23]:
import re
merged_applicant_dfs = []
for df, df_name in dataframes:
    if re.match("award", df_name):
        continue
    print("-----", df_name)
    num_of_matches = [0] * 20
    for i, col in enumerate(df.columns):
        if re.match("average", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "avg_targeted_affordability"})
            num_of_matches[0] += 1
            # print([col for col in df.columns if re.search("avg", col, re.IGNORECASE)])
        elif re.match("CDLAC TOTAL", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "total_points"})
            num_of_matches[1] += 1
            # print([col for col in df.columns if re.search("CDLAC_total", col, re.IGNORECASE)])
        elif re.search("tie-brea", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "tie_breaker_self_score"})
            num_of_matches[2] += 1
            # print([col for col in df.columns if re.search("tie_brea", col, re.IGNORECASE)])
        elif re.search("bond", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "bond_request_amount"})
            num_of_matches[3] += 1
            # print([col for col in df.columns if re.search("bond", col, re.IGNORECASE)])
        elif re.search("units for homeless", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "num_homeless_units"})
            num_of_matches[4] += 1
            # print([col for col in df.columns if re.search("homeless_units", col, re.IGNORECASE)])
        elif re.search("construction type", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "construction_type"})
            num_of_matches[5] += 1
            # print([col for col in df.columns if re.search("construction_type", col, re.IGNORECASE)])
        elif re.search("housing type", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "housing_type"})
            num_of_matches[6] += 1
            # print([col for col in df.columns if re.search("housing_type", col, re.IGNORECASE)])
        elif re.search("CDLAC.*region", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "CDLAC_region"})
            num_of_matches[7] += 1
            # print([col for col in df.columns if re.search("CDLAC_region", col, re.IGNORECASE)])
        elif re.search("CDLAC.*pool", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "CDLAC_pool"})
            num_of_matches[8] += 1
            # print([col for col in df.columns if re.search("CDLAC_pool", col, re.IGNORECASE)])
        elif re.search("BIPOC", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "bipoc_binary"})
            num_of_matches[9] += 1
            # print([col for col in df.columns if re.search("bipoc_binary", col, re.IGNORECASE)])
        elif re.match("new construction set aside", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "new_construction_set_aside"})
            num_of_matches[10] += 1
            # print([col for col in df.columns if re.search("new_construction", col, re.IGNORECASE)])
        elif re.search("secondary new construction", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "secondary_new_construction_set_aside"})
            num_of_matches[11] += 1
            # print([col for col in df.columns if re.search("secondary_new", col, re.IGNORECASE)])
        elif re.search("application", col, re.IGNORECASE):
            # print(col)
            df = df.rename(columns={col: "application_number"})
            num_of_matches[12] += 1
            # print([col for col in df.columns if re.search("application", col, re.IGNORECASE)])
    # ensure that each regular expression only matches to one column in each dataframe
    for i in range(20):
        assert(num_of_matches[i] < 2) 
    
    # Drop rows where at least 90% of columns are NaN
    threshold = int(len(df.columns) * 0.1)  # Adjust threshold as needed (e.g., 90% of columns)
    df = df.dropna(thresh=threshold)

    # do some merging and some cleaning  
    if "secondary_new_construction_set_aside" in df.columns:
        # print("  before: ", df['new_construction_set_aside'].value_counts())
        # Handle NaN/None by converting to empty strings
        df['new_construction_set_aside'] = df['new_construction_set_aside'].fillna('')
        df['secondary_new_construction_set_aside'] = df['secondary_new_construction_set_aside'].fillna('')
        # Standardize capitalization (e.g., convert to uppercase)
        df['new_construction_set_aside'] = df['new_construction_set_aside'].str.upper()
        df['secondary_new_construction_set_aside'] = df['secondary_new_construction_set_aside'].str.upper()
        # Concatenate, avoiding extra commas for empty secondary values
        df['combined_set_aside'] = (
            df['new_construction_set_aside'] +
            df['secondary_new_construction_set_aside'].apply(lambda x: f", {x}" if x else "")
        )
        # Remove leading/trailing commas and extra spaces
        df['combined_set_aside'] = df['combined_set_aside'].str.strip(", ")
        # Replace empty strings with None in a specific column
        df['combined_set_aside'] = df['combined_set_aside'].replace('', None)
        # print(" after: ", df['combined_set_aside'].value_counts())
    else:
        # Standardize capitalization (e.g., convert to uppercase)
        df['combined_set_aside'] = df['new_construction_set_aside'].str.upper()
        # print("  before and after: ", df['new_construction_set_aside'].value_counts())
    
    # print(" before: ", df['CDLAC_pool'].value_counts())
    df['CDLAC_pool'] = df['CDLAC_pool'].str.upper()
    df['CDLAC_pool'] = df['CDLAC_pool'].str.strip(", ")
    if "bipoc_binary" in df.columns:
        df['combined_CDLAC_pool'] = np.where(df['bipoc_binary'] == 'Yes', 'BIPOC', df['CDLAC_pool'])
        # print(" after: ", df['combined_CDLAC_pool'].value_counts())
    else:
        df['combined_CDLAC_pool'] = df['CDLAC_pool']
        # print(" after: ", df['combined_CDLAC_pool'].value_counts())
    
    
    # print(df['construction_type'].value_counts()) 
    # print(df['housing_type'].value_counts())
    new_columns = [
        "application_number",
        "avg_targeted_affordability",
        "total_points",
        "tie_breaker_self_score",
        "bond_request_amount",
        "num_homeless_units",
        "construction_type",
        "housing_type",
        "CDLAC_region",
        "combined_CDLAC_pool",
        "combined_set_aside",
    ]
    merged_applicant_dfs.append((df[new_columns], df_name))



----- R1_2023_applicant
----- R2_2023_applicant
----- R3_2023_applicant
----- R1_2024_applicant
----- R2_2024_applicant
----- R1_2025_applicant


In [24]:
datapoints = 0
for df, df_name in merged_applicant_dfs:
    print("-------", df_name, df.shape)
    datapoints += df.shape[0]
print("total number of data points over 2.5 years is: ", datapoints)

# merge all dfs into one and save in csv

applicant_df = pd.DataFrame()
for df, df_name in merged_applicant_dfs:
    print("-------", df_name)
    applicant_df = pd.concat([applicant_df, df])
print(applicant_df.shape)

    

------- R1_2023_applicant (85, 11)
------- R2_2023_applicant (60, 11)
------- R3_2023_applicant (79, 11)
------- R1_2024_applicant (161, 11)
------- R2_2024_applicant (193, 11)
------- R1_2025_applicant (91, 11)
total number of data points over 2.5 years is:  669
------- R1_2023_applicant
------- R2_2023_applicant
------- R3_2023_applicant
------- R1_2024_applicant
------- R2_2024_applicant
------- R1_2025_applicant
(669, 11)


In [27]:
applicant_df.to_csv("../data/interim/merged_applicant_3yr.csv", index=False)